In [1]:
from __future__ import division
from sklearn.preprocessing import CategoricalEncoder
from sklearn.preprocessing import LabelEncoder

import torchvision.models as models
from multiprocessing import Pool
import time
import numpy as np
import pandas as pd
import datetime as dt
import os.path as dpath
import matplotlib.pyplot as plt
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from sklearn.model_selection import train_test_split
from mpl_toolkits.axes_grid1 import ImageGrid
from PIL import Image
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models
import pdb
from pathlib import Path
import torch.optim as optim


import torch.nn as nn
import torch.nn.functional as F

from IPython.display import FileLink, FileLinks


%matplotlib inline

from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

In [2]:
path = "/home/edwin/Datasets/competitions/avito-demand-prediction/"
PATH = Path(path)

In [3]:
TRN_CSV = pd.read_csv(PATH/'train.csv')
TRN_CSV_COPY = TRN_CSV.copy()

TEST_CSV = pd.read_csv(PATH/'test.csv')
TEST_CSV_COPY = TEST_CSV.copy()

In [4]:
cat_vars = ['item_id', 'user_id', 'region', 'city', 'parent_category_name', 'category_name', 
           'param_1', 'param_2', 'param_3', 'title', 'description', 'user_type']
cont_vars = ['price', 'item_seq_number', 'deal_probability']
cont_vars_test = ['price', 'item_seq_number']
skip_vars = ['activation_date', 'image']


In [5]:
n = len(TRN_CSV_COPY); n

1503424

In [6]:
n_test = len(TEST_CSV_COPY); n_test

508438

In [7]:
TEST_CSV_COPY['deal_probability'] = [0 for i in range(n_test)]

In [8]:
for c in cat_vars:
    TRN_CSV_COPY[c] = TRN_CSV_COPY[c].astype("category").cat.as_ordered()
    TEST_CSV_COPY[c] = TEST_CSV_COPY[c].astype("category").cat.as_ordered()


In [9]:
for c in cont_vars:
    TRN_CSV_COPY[c] = TRN_CSV_COPY[c].astype("float32")
    TEST_CSV_COPY[c] = TEST_CSV_COPY[c].astype("float32")

In [10]:
idxs = get_cv_idxs(n, val_pct=15000/n)

In [11]:
TRN_SAMPLE = TRN_CSV_COPY.iloc[idxs]

In [12]:
n_sample = len(TRN_SAMPLE)

In [13]:
n_sample

15000

In [14]:
train_ratio = 0.75

In [15]:
train_size = int(n_sample * train_ratio); train_size

11250

In [16]:
df, y, nas, mapper = proc_df(TRN_SAMPLE, 'deal_probability', do_scale=True, skip_flds=skip_vars)

In [17]:
df

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,user_type,image_top_1,price_na,image_top_1_na
143309,1111769,325705,11,744,5,30,41,0,0,685435,1083896,-0.046974,-0.115768,2,-1.034376,-0.243715,-0.292876
771572,293470,460746,28,1725,5,43,38,0,0,109900,601767,-0.053180,-0.061905,1,-0.453284,-0.243715,-0.292876
1378472,1277167,345127,24,1542,5,30,207,201,164,501637,1047537,-0.057483,-0.129318,2,-0.605013,-0.243715,-0.292876
846158,1129361,183646,13,1049,5,11,105,193,105,637144,1004675,-0.057870,-0.032603,1,-1.211932,-0.243715,-0.292876
84567,896590,767860,10,1043,5,30,126,203,139,784059,1183459,-0.057992,-0.127963,2,-1.222693,-0.243715,-0.292876
984506,1447298,426696,17,1277,5,30,126,203,139,445539,572608,-0.057704,-0.119833,2,-0.875113,-0.243715,-0.292876
1276450,1309392,588499,24,1542,1,42,13,0,0,410853,1055306,-0.057041,5.498986,3,1.856020,-0.243715,-0.292876
1063503,74820,163965,20,461,5,30,41,0,0,84109,974463,-0.057815,-0.132367,1,1.128579,-0.243715,-0.292876
170088,1254217,741249,15,1108,6,16,258,2,1174,14642,115724,0.305894,-0.131520,2,0.125657,-0.243715,-0.292876
1334881,454169,226900,20,461,5,30,126,232,144,642211,993559,-0.057593,-0.119664,1,-0.709395,-0.243715,-0.292876


In [18]:
val_idxs = list(range(train_size, len(df)))

In [19]:
df_test, _, nas, mapper = proc_df(TEST_CSV_COPY, 'deal_probability', do_scale=True, 
                                  skip_flds=skip_vars, mapper=mapper, na_dict=nas)

In [20]:
len(df_test)

508438

In [21]:
md = ColumnarModelData.from_data_frame(path, val_idxs, df, y.astype(np.float32), 
                                       cat_flds=cat_vars, bs=4, test_df=df_test)

In [22]:
cat_sz = [(c, len(TRN_CSV_COPY[c].cat.categories)+1) for c in cat_vars]; cat_sz

[('item_id', 1503425),
 ('user_id', 771770),
 ('region', 29),
 ('city', 1734),
 ('parent_category_name', 10),
 ('category_name', 48),
 ('param_1', 372),
 ('param_2', 272),
 ('param_3', 1220),
 ('title', 788378),
 ('description', 1317103),
 ('user_type', 4)]

In [23]:
emb_szs = [(c, min(50, (c+1)//2)) for _, c in cat_sz]; emb_szs

[(1503425, 50),
 (771770, 50),
 (29, 15),
 (1734, 50),
 (10, 5),
 (48, 24),
 (372, 50),
 (272, 50),
 (1220, 50),
 (788378, 50),
 (1317103, 50),
 (4, 2)]

In [24]:
len(df)

15000

In [25]:
max_y = np.max(y); max_y

1.0

In [26]:
y_range = (0,1)

In [33]:
df

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,user_type,image_top_1,price_na,image_top_1_na
143309,1111769,325705,11,744,5,30,41,0,0,685435,1083896,-0.046974,-0.115768,2,-1.034376,-0.243715,-0.292876
771572,293470,460746,28,1725,5,43,38,0,0,109900,601767,-0.053180,-0.061905,1,-0.453284,-0.243715,-0.292876
1378472,1277167,345127,24,1542,5,30,207,201,164,501637,1047537,-0.057483,-0.129318,2,-0.605013,-0.243715,-0.292876
846158,1129361,183646,13,1049,5,11,105,193,105,637144,1004675,-0.057870,-0.032603,1,-1.211932,-0.243715,-0.292876
84567,896590,767860,10,1043,5,30,126,203,139,784059,1183459,-0.057992,-0.127963,2,-1.222693,-0.243715,-0.292876
984506,1447298,426696,17,1277,5,30,126,203,139,445539,572608,-0.057704,-0.119833,2,-0.875113,-0.243715,-0.292876
1276450,1309392,588499,24,1542,1,42,13,0,0,410853,1055306,-0.057041,5.498986,3,1.856020,-0.243715,-0.292876
1063503,74820,163965,20,461,5,30,41,0,0,84109,974463,-0.057815,-0.132367,1,1.128579,-0.243715,-0.292876
170088,1254217,741249,15,1108,6,16,258,2,1174,14642,115724,0.305894,-0.131520,2,0.125657,-0.243715,-0.292876
1334881,454169,226900,20,461,5,30,126,232,144,642211,993559,-0.057593,-0.119664,1,-0.709395,-0.243715,-0.292876


In [27]:
df.columns

Index(['item_id', 'user_id', 'region', 'city', 'parent_category_name',
       'category_name', 'param_1', 'param_2', 'param_3', 'title',
       'description', 'price', 'item_seq_number', 'user_type', 'image_top_1',
       'price_na', 'image_top_1_na'],
      dtype='object')

In [30]:
cat_vars

['item_id',
 'user_id',
 'region',
 'city',
 'parent_category_name',
 'category_name',
 'param_1',
 'param_2',
 'param_3',
 'title',
 'description',
 'user_type']

In [28]:
len(cat_vars)

12

In [29]:
len(df.columns)-len(cat_vars)

5

In [34]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars), 0.04, 1, [1000,500], [0.001, 0.01], y_range=y_range)

In [ ]:
m

In [ ]:
lr = 1e-3

In [35]:
m.lr_find()

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

  0%|          | 0/2813 [00:00<?, ?it/s]


RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1522182087074/work/torch/lib/THC/generic/THCStorage.cu:58

In [39]:
lr = 1e-5

In [40]:
m.fit(lr, 3)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

  0%|          | 0/2813 [00:00<?, ?it/s]


RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1522182087074/work/torch/lib/THC/THCTensorCopy.cu:204

In [41]:
m

MixedInputModel(
  (embs): ModuleList(
    (0): Embedding(1503425, 50)
    (1): Embedding(771770, 50)
    (2): Embedding(29, 15)
    (3): Embedding(1734, 50)
    (4): Embedding(10, 5)
    (5): Embedding(48, 24)
    (6): Embedding(372, 50)
    (7): Embedding(272, 50)
    (8): Embedding(1220, 50)
    (9): Embedding(788378, 50)
    (10): Embedding(1317103, 50)
    (11): Embedding(4, 2)
  )
  (lins): ModuleList(
    (0): Linear(in_features=451, out_features=1000, bias=True)
    (1): Linear(in_features=1000, out_features=500, bias=True)
  )
  (bns): ModuleList(
    (0): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True)
    (1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True)
  )
  (outp): Linear(in_features=500, out_features=1, bias=True)
  (emb_drop): Dropout(p=0.04)
  (drops): ModuleList(
    (0): Dropout(p=0.001)
    (1): Dropout(p=0.01)
  )
  (bn): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True)
)

In [43]:
dir(md)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'from_arrays',
 'from_data_frame',
 'from_data_frames',
 'from_dls',
 'get_learner',
 'is_reg',
 'path',
 'test_dl',
 'test_ds',
 'trn_dl',
 'trn_ds',
 'trn_y',
 'val_dl',
 'val_ds',
 'val_y']

In [104]:
md.trn_ds[0][0]

array([1111769,  325705,      11,     744,       5,      30,      41,       0,       0,  685435, 1083896,
             2])

In [60]:
x_cat = md.trn_ds[0][0]

In [61]:
x_cont = md.trn_ds[0][1]

In [73]:
x_cat, x_cont, y = next(iter(md.trn_dl))

4

In [57]:
md.trn_y[0]

array([ 0.], dtype=float32)

In [56]:
md.test_ds[0]

[array([201001, 262939,      5,    307,      5,     11,    108,    177,     63, 181529, 153640,      2]),
 array([-0.05774, -0.12136,  0.86386,  4.10316, -0.29288], dtype=float32),
 array([ 0.])]

In [59]:
m.model

MixedInputModel(
  (embs): ModuleList(
    (0): Embedding(1503425, 50)
    (1): Embedding(771770, 50)
    (2): Embedding(29, 15)
    (3): Embedding(1734, 50)
    (4): Embedding(10, 5)
    (5): Embedding(48, 24)
    (6): Embedding(372, 50)
    (7): Embedding(272, 50)
    (8): Embedding(1220, 50)
    (9): Embedding(788378, 50)
    (10): Embedding(1317103, 50)
    (11): Embedding(4, 2)
  )
  (lins): ModuleList(
    (0): Linear(in_features=451, out_features=1000, bias=True)
    (1): Linear(in_features=1000, out_features=500, bias=True)
  )
  (bns): ModuleList(
    (0): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True)
    (1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True)
  )
  (outp): Linear(in_features=500, out_features=1, bias=True)
  (emb_drop): Dropout(p=0.04)
  (drops): ModuleList(
    (0): Dropout(p=0.001)
    (1): Dropout(p=0.01)
  )
  (bn): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True)
)

In [64]:
Variable(torch.from_numpy(x_cat))

Variable containing:
 1.1118e+06
 3.2570e+05
 1.1000e+01
 7.4400e+02
 5.0000e+00
 3.0000e+01
 4.1000e+01
 0.0000e+00
 0.0000e+00
 6.8544e+05
 1.0839e+06
 2.0000e+00
[torch.LongTensor of size 12]

In [65]:
Variable(torch.from_numpy(x_cont))

Variable containing:
-0.0470
-0.1158
-1.0344
-0.2437
-0.2929
[torch.FloatTensor of size 5]

In [80]:
m.model(Variable(x_cat), Variable(x_cont))

TypeError: torch.index_select received an invalid combination of arguments - got ([32;1mtorch.cuda.FloatTensor[0m, [32;1mint[0m, [31;1mtorch.LongTensor[0m), but expected (torch.cuda.FloatTensor source, int dim, torch.cuda.LongTensor index)

In [87]:
next(iter(md.trn_dl))

[
 
 Columns 0 to 5 
  4.4040e+05  7.2348e+05  1.7000e+01  1.2770e+03  1.0000e+00  4.5000e+01
  1.3917e+06  2.2322e+05  1.3000e+01  1.0490e+03  1.0000e+00  3.1000e+01
  3.9159e+05  2.2800e+05  2.7000e+01  1.6330e+03  1.0000e+00  2.8000e+01
  1.4017e+06  1.8173e+05  2.3000e+01  1.6930e+03  5.0000e+00  3.0000e+01
 
 Columns 6 to 11 
  7.8000e+01  0.0000e+00  0.0000e+00  1.6410e+05  7.0061e+05  2.0000e+00
  2.6500e+02  1.5500e+02  0.0000e+00  2.8192e+05  1.2626e+06  2.0000e+00
  0.0000e+00  0.0000e+00  0.0000e+00  4.3904e+05  9.5230e+05  1.0000e+00
  1.2600e+02  2.3200e+02  1.4400e+02  6.4227e+05  1.1516e+06  2.0000e+00
 [torch.LongTensor of size 4x12], 
 -0.0580 -0.1178  1.7398 -0.2437 -0.2929
 -0.0578 -0.1305  1.6806 -0.2437 -0.2929
 -0.0573 -0.1181  1.8732 -0.2437 -0.2929
 -0.0573 -0.1312 -0.7072 -0.2437 -0.2929
 [torch.FloatTensor of size 4x5], 
  0.0000
  0.0000
  0.3246
  0.0000
 [torch.FloatTensor of size 4x1]]

In [88]:
x_cat, x_cont, y = next(iter(md.trn_dl))

In [89]:
x_cat



Columns 0 to 5 
 7.2742e+05  5.2266e+05  2.2000e+01  5.8600e+02  8.0000e+00  3.5000e+01
 9.9214e+05  1.2332e+05  2.0000e+01  4.6100e+02  5.0000e+00  3.0000e+01
 3.3169e+05  5.6759e+05  1.6000e+01  1.1540e+03  6.0000e+00  1.4000e+01
 1.2928e+06  1.0392e+05  1.0000e+00  1.2400e+02  5.0000e+00  3.0000e+01

Columns 6 to 11 
 2.6900e+02  2.6800e+02  0.0000e+00  2.5622e+05  1.2115e+06  2.0000e+00
 1.2600e+02  1.9300e+02  1.1700e+02  5.5357e+05  5.4308e+05  2.0000e+00
 2.5800e+02  2.0800e+02  0.0000e+00  7.2841e+05  1.1293e+06  1.0000e+00
 1.2600e+02  1.9300e+02  1.1500e+02  2.0214e+05  7.3034e+05  2.0000e+00
[torch.LongTensor of size 4x12]

In [90]:
x_cont


-0.0580 -0.1317  1.6634 -0.2437 -0.2929
-0.0579 -0.1308 -0.7836 -0.2437 -0.2929
-0.0523 -0.1200 -0.1735 -0.2437  3.4144
-0.0576 -0.1015 -0.7836 -0.2437 -0.2929
[torch.FloatTensor of size 4x5]

In [91]:
y


1.00000e-02 *
  0.0000
  0.0000
  5.5570
  0.0000
[torch.FloatTensor of size 4x1]

In [93]:
m.model.forward

<bound method MixedInputModel.forward of MixedInputModel(
  (embs): ModuleList(
    (0): Embedding(1503425, 50)
    (1): Embedding(771770, 50)
    (2): Embedding(29, 15)
    (3): Embedding(1734, 50)
    (4): Embedding(10, 5)
    (5): Embedding(48, 24)
    (6): Embedding(372, 50)
    (7): Embedding(272, 50)
    (8): Embedding(1220, 50)
    (9): Embedding(788378, 50)
    (10): Embedding(1317103, 50)
    (11): Embedding(4, 2)
  )
  (lins): ModuleList(
    (0): Linear(in_features=451, out_features=1000, bias=True)
    (1): Linear(in_features=1000, out_features=500, bias=True)
  )
  (bns): ModuleList(
    (0): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True)
    (1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True)
  )
  (outp): Linear(in_features=500, out_features=1, bias=True)
  (emb_drop): Dropout(p=0.04)
  (drops): ModuleList(
    (0): Dropout(p=0.001)
    (1): Dropout(p=0.01)
  )
  (bn): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True)
)>

In [95]:
model = m.model

In [97]:
[e for i, e in enumerate(model.embs)]

[Embedding(1503425, 50),
 Embedding(771770, 50),
 Embedding(29, 15),
 Embedding(1734, 50),
 Embedding(10, 5),
 Embedding(48, 24),
 Embedding(372, 50),
 Embedding(272, 50),
 Embedding(1220, 50),
 Embedding(788378, 50),
 Embedding(1317103, 50),
 Embedding(4, 2)]

In [98]:
len(x_cat)

4

In [102]:
[e(x_cat[:,i]) for i, e in enumerate(model.embs)]

TypeError: torch.index_select received an invalid combination of arguments - got ([32;1mtorch.cuda.FloatTensor[0m, [32;1mint[0m, [31;1mtorch.LongTensor[0m), but expected (torch.cuda.FloatTensor source, int dim, torch.cuda.LongTensor index)

In [103]:
x_cat



Columns 0 to 5 
 7.2742e+05  5.2266e+05  2.2000e+01  5.8600e+02  8.0000e+00  3.5000e+01
 9.9214e+05  1.2332e+05  2.0000e+01  4.6100e+02  5.0000e+00  3.0000e+01
 3.3169e+05  5.6759e+05  1.6000e+01  1.1540e+03  6.0000e+00  1.4000e+01
 1.2928e+06  1.0392e+05  1.0000e+00  1.2400e+02  5.0000e+00  3.0000e+01

Columns 6 to 11 
 2.6900e+02  2.6800e+02  0.0000e+00  2.5622e+05  1.2115e+06  2.0000e+00
 1.2600e+02  1.9300e+02  1.1700e+02  5.5357e+05  5.4308e+05  2.0000e+00
 2.5800e+02  2.0800e+02  0.0000e+00  7.2841e+05  1.1293e+06  1.0000e+00
 1.2600e+02  1.9300e+02  1.1500e+02  2.0214e+05  7.3034e+05  2.0000e+00
[torch.LongTensor of size 4x12]

In [105]:
x_cont


-0.0580 -0.1317  1.6634 -0.2437 -0.2929
-0.0579 -0.1308 -0.7836 -0.2437 -0.2929
-0.0523 -0.1200 -0.1735 -0.2437  3.4144
-0.0576 -0.1015 -0.7836 -0.2437 -0.2929
[torch.FloatTensor of size 4x5]

In [106]:
e = model.embs[0]

In [117]:
e()

TypeError: torch.index_select received an invalid combination of arguments - got ([32;1mtorch.cuda.FloatTensor[0m, [32;1mint[0m, [31;1mtorch.LongTensor[0m), but expected (torch.cuda.FloatTensor source, int dim, torch.cuda.LongTensor index)

In [120]:
e(Variable(x_cat)[:,0])

TypeError: torch.index_select received an invalid combination of arguments - got ([32;1mtorch.cuda.FloatTensor[0m, [32;1mint[0m, [31;1mtorch.LongTensor[0m), but expected (torch.cuda.FloatTensor source, int dim, torch.cuda.LongTensor index)

In [115]:
e

Embedding(1503425, 50)